In [1]:
from pylablib.devices import Thorlabs
from scipy.fftpack import rfft, irfft, fftfreq
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import MatplotlibDeprecationWarning
import warnings
import os.path
from pathlib import Path
import Oscilloscope
from InquirerPy import inquirer
from InquirerPy.validator import PathValidator
import matplotlib.gridspec as gridspec
import keyboard
import time
import math
from itertools import combinations
import Validators as vd

In [2]:

result = {}
filename = 'backup files/ColorGlass_old.txt'


data = np.loadtxt(filename,skiprows=1)
header = open(filename).readline()


data = remove_zeros(data)
data = calc_od(data)
filter_titles = header.split('\n')[0].split('\t')[2:]


wl_index = np.where(data[1:,0] == wavelength)[0][0] + 1

filter_dict = {}
for key, value in zip(filter_titles,data[wl_index,2:]):
    filter_dict.update({key:value})

filter_combinations = {}
for i in range(max_combinations):
    for comb in combinations(filter_dict.items(),i+1):
        key = ''
        value = 0
        for k,v in comb:
            key +=k
            value+=v
        filter_combinations.update({key:math.pow(10,-value)})    

target_energy = target_energy*1000
laser_energy = current_energy_pm/data[wl_index,1]*100

target_transm = target_energy/laser_energy
if not no_print:
    print(f'Target energy = {target_energy} [uJ]')
    print(f'Current laser output = {laser_energy:.0f} [uJ]')
    print(f'Target transmission = {target_transm*100:.1f} %')
    print(f'{bcolors.HEADER} Valid filter combinations:{bcolors.ENDC}')
filter_combinations = dict(sorted(filter_combinations.copy().items(), key=lambda item: item[1]))

i=0
for key, value in filter_combinations.items():
    if (value-target_transm) > 0 and value/target_transm < 2.5:
        result.update({key: value})
        if not no_print:
            if (value/target_transm < 1.25) and i<5:
                print(f'{bcolors.OKGREEN} {key}, transmission = {value*100:.1f}%{bcolors.ENDC} (target= {target_transm*100:.1f}%)')
            elif (value/target_transm < 1.5) and i<5:
                print(f'{bcolors.OKCYAN} {key}, transmission = {value*100:.1f}%{bcolors.ENDC} (target= {target_transm*100:.1f}%)')
            elif value/target_transm < 2 and i<5:
                print(f'{bcolors.OKBLUE} {key}, transmission = {value*100:.1f}%{bcolors.ENDC} (target= {target_transm*100:.1f}%)')
            elif value/target_transm < 2.5 and i<5:
                print(f'{bcolors.WARNING} {key}, transmission = {value*100:.1f}%{bcolors.ENDC} (target= {target_transm*100:.1f}%)')
        i+=1

if not no_print:
    print('\n')

target_pm_value = target_energy*data[wl_index,1]/100



NameError: name 'remove_zeros' is not defined